In [ ]:
#use this ipynb to create a dictionary for YF mechanism 

In [1]:
import rmgpy.chemkin
import os
import re
import numpy as np
from rmgpy.rmg.main import RMG, RMG_Memory, log_conditions
import rmgpy.chemkin
from rmgpy.rmg.model import ReactionModel, CoreEdgeReactionModel
from rmgpy import settings
from rmgpy.data.rmg import RMGDatabase
from rmgpy.molecule import Molecule

In [ ]:
#first, read in the markdown chemkin file
with open('YF_Mech_Final.m', 'r') as f:
    lines = f.readlines()

In [ ]:
#get the entire species part of the chemkin
species_segment = lines[36:248]

#toss out the lines we don't want
species_lines = []
for line in species_segment: 
    if line[0]=='!' or line=='\n': 
        print(line)
    else:
        species_lines.append(line)

        

In [ ]:
spec_dict_str = []


# get all the lines with InChI
no_InChI_lines = []
InChI_lines = []
for line in species_lines: 
    if 'InChI' in line: 
        InChI_lines.append(line)
    else:
        no_InChI_lines.append(line)
        

# now go through all the InChI lines and match the species names to the InChi
problematic_InChI = []
for line in InChI_lines: 
    try:
        species_name = re.search('^(\S+)\s+', line).group(1)
        InChI = re.search('(InChI=\S+)\n', line).group(1)
        mole = Molecule().from_inchi(inchistr=InChI)    
        adj_list = mole.to_adjacency_list()
        spec_dict_str.append(species_name+'\n'+adj_list)
    except:
        problematic_InChI.append(line)
        pass
    
    

In [ ]:
species_with_no_InChI= []

for line in no_InChI_lines: 
    edited_line = line.replace('\n','   ').replace('\t','   ')
    #grab the first species
    try: 
        #get the species at beginning of line 
        species = re.search('^([A-Z0-9]+) ',edited_line).group(1)
        species_with_no_InChI.append(species)
        #now see if there's any more in the rest of the line
        
        #for the species that look regular (CF2, etc). 
        species_list = re.findall(' ([A-Z0-9]+) ',edited_line)
        species_with_no_InChI.extend(species_list)
        
        #for the species that have a dash in the middle (CH3-CF3)
        species_list = re.findall(' ([A-Z0-9]+-[A-Z0-9]+) ',edited_line)
        species_with_no_InChI.extend(species_list)
        
        #for the species with a : in the middle (CHF:CF2)
        species_list = re.findall(' ([A-Z0-9]+:[A-Z0-9]+) ',edited_line)
        species_with_no_InChI.extend(species_list)        

    except AttributeError as e: 
        print(edited_line)
        pass

In [ ]:
print(len(spec_dict_str))
print(len(problematic_InChI))
print(len(InChI_lines), len(no_InChI_lines))

In [ ]:
add_these = ['CH2*', 'nC4H3', 'iC4H3','C4H5-2', 'CHF','CF2','CF', 'CHF:O',
             'CH3-CH2F',  'CH3-CHF2' , 'CH3-CF3',
             'CH2F-CH2F', 'CH2F-CHF2', 'CH2F-CF3', 'CHF2-CHF2' ,'CHF2-CF3', 'CHF2-CHF2', 'CHF2-CF3',
             'CF3-CF3','CH3-CHF', 'CH3-CF2',
 'CH2F-CH2' , 'CH2F-CHF' , 'CH2F-CF2',
 'CHF2-CH2' , 'CHF2-CHF' , 'CHF2-CF2',
 'CF3-CH2' ,  'CF3-CHF'  , 'CF3-CF2',       
 'CH2:CHF',    'CH2:CF2',
 'CHF:CHF[Z]', 'CHF:CF2',
 'CF2:CF2\n','CH2:CF',
 'CHF:CH[Z]', 'CHF:CF[Z]',
 'CF2:CH' ,   'CF2:CF']

species_with_no_InChI.extend(add_these)

In [ ]:
spec_dict_str